## Installation Dependencies

In [20]:
%%capture
!pip install --upgrade -qqq uv
!pip install peft boto3 trl datasets
!pip install -qU unsloth unsloth_zoo
try: import numpy; install_numpy = f"numpy=={numpy.__version__}"
except: install_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {install_numpy} \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    torchvision bitsandbytes \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels \
    "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git

## Load Model

In [21]:
from unsloth import FastLanguageModel
import torch

# Load an unsloth model
max_seq_length = 4096
dtype = None
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    full_finetuning = False,
)

==((====))==  Unsloth 2025.8.9: Fast Gpt_Oss patching. Transformers: 4.55.4.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.529 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
# Param efficient + LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


## Load Dataset

In [ ]:
import boto3
import json
import pprint

# Create a dataset
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

s3 = boto3.client('s3', aws_access_key_id="",
                  aws_secret_access_key="")
END = "<|end|>"

def concatenate_json_data(bucket_name: str, prefix: str = '') -> list:
    all_data = []

    try:
        paginator = s3.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

        for page in pages:
            if "Contents" in page:
                for obj in page['Contents']:
                    if obj['Key'].endswith('.json'):
                        try:
                            response = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
                            content = response['Body'].read().decode('utf-8')
                            data = json.loads(content)
                            if isinstance(data.get('data'), list):
                              for item in data['data']:
                                if item["validation_result"]["is_valid"]:
                                  all_data.append({
                                      "text": f"Input: {item['processed']}\nOutput: {item['original']} {END}"
                                  })
                            else:
                                all_data.extend(data['data'])
                        except Exception as e:
                            print(f"Skipping file {obj['Key']} due to error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    return all_data

dataset = concatenate_json_data("fb-cleaned-data")

In [24]:
from datasets import Dataset
dataset = Dataset.from_list(dataset)

# Load Trainer

In [25]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

# Create a trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        num_train_epochs = 1,
        learning_rate = 2e-4,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        completion_only_loss=True
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4631 [00:00<?, ? examples/s]

In [ ]:
# Perform the fine tuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,631 | Num Epochs = 1 | Total steps = 579
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Step,Training Loss
1,5.148100
2,5.437000
3,5.168900
4,5.216800
5,4.056000
6,4.862600
7,4.491000
8,4.366100
9,4.907700
10,4.134700


Unsloth: Will smartly offload gradients to save VRAM!


## Inference

In [ ]:
FastLanguageModel.for_inference(model)

# Run inference
text = "So tell me, what else is going on"
prompt = f"Input: {text}.\nOutput:"

enc = tokenizer(prompt, return_tensors="pt").to(model.device)
generator = model.generate(
    **enc,
    max_new_tokens=48,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
input_length = enc["input_ids"].shape[-1]
completion = tokenizer.decode(generator[0, input_length:], skip_special_tokens=True)
completion = completion.split(END)[0].strip().split("\n")[0]
print(completion)

## Save to S3

In [ ]:
# Save locally
model.save_pretrained("venkat_ft_model")
tokenizer.save_pretrained("venkat_ft_tokenizer")

# Upload
import boto3
import os
import time

def upload_to_s3(local_file_path, s3_file_path):
    try:
        # Upload the file to S3
        s3.upload_file(local_file_path, "venkat-ft-artifacts", s3_file_path)
        print("Upload successful!")
        return True
    except Exception as e:
        print(f"Upload failed: {e}")
        return False


# Write every file, timestamped
for file_name in os.listdir("venkat_ft_model"):
    epoch_time = str(int(time.time()))
    output_file_name = epoch_time + "_" + file_name
    file_path = "venkat_ft_model/" + file_name
    upload_to_s3(file_path, output_file_name)

for file_name in os.listdir("venkat_ft_tokenizer"):
    epoch_time = str(int(time.time()))
    output_file_name = epoch_time + "_" + file_name
    file_path = "venkat_ft_tokenizer/" + file_name
    upload_to_s3(file_path, output_file_name)